<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Useful-Scripts" data-toc-modified-id="Useful-Scripts-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Useful Scripts</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Random-Forest-Modelling" data-toc-modified-id="Random-Forest-Modelling-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Random Forest Modelling</a></span></li><li><span><a href="#Grid-Search" data-toc-modified-id="Grid-Search-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Grid Search</a></span></li><li><span><a href="#Use-the-best-parameters-from-grid-search" data-toc-modified-id="Use-the-best-parameters-from-grid-search-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Use the best parameters from grid search</a></span></li><li><span><a href="#Randomized-search" data-toc-modified-id="Randomized-search-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Randomized search</a></span></li><li><span><a href="#Feature-Importance" data-toc-modified-id="Feature-Importance-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Feature Importance</a></span></li><li><span><a href="#Feature-Importance-using-eli5" data-toc-modified-id="Feature-Importance-using-eli5-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Feature Importance using eli5</a></span></li></ul></div>

<div class="alert alert-block alert-success">
<b>Kernel Author:</b>  <br>
<a href="https://bhishanpdl.github.io/" , target="_blank">Bhishan Poudel,  Data Scientist, Ph.D Astrophysics</a> .
</div>

# Imports

In [1]:
import time
time_start_notebook = time.time()

# usual imports
import numpy as np
import pandas as pd

import os
import time
import collections
import itertools
import six
import pickle
import joblib

# random state
SEED=100
np.random.seed(SEED) # we need this in each cell

# sklearn
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

# versions
import watermark
%load_ext watermark
%watermark -a "Bhishan Poudel" -d -v -m
print()
%watermark -iv

Bhishan Poudel 2020-11-02 

CPython 3.7.7
IPython 7.18.1

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 19.6.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit

six       1.15.0
sklearn   0.23.1
watermark 2.0.2
numpy     1.18.4
pandas    1.1.0
joblib    0.17.0



# Useful Scripts

In [2]:
def show_methods(obj, ncols=7,start=None, inside=None):
    """ Show all the attributes of a given method.
    Example:
    ========
    show_method_attributes(list)
     """
    lst = [elem for elem in dir(obj) if elem[0]!='_' ]
    lst = [elem for elem in lst 
           if elem not in 'os np pd sys time psycopg2'.split() ]

    if isinstance(start,str):
        lst = [elem for elem in lst if elem.startswith(start)]
        
    if isinstance(start,tuple) or isinstance(start,list):
        lst = [elem for elem in lst for start_elem in start
               if elem.startswith(start_elem)]
        
    if isinstance(inside,str):
        lst = [elem for elem in lst if inside in elem]
        
    if isinstance(inside,tuple) or isinstance(inside,list):
        lst = [elem for elem in lst for inside_elem in inside
               if inside_elem in elem]

    return pd.DataFrame(np.array_split(lst,ncols)).T.fillna('')

def adjustedR2(rsquared,nrows,kcols):
    return rsquared- (kcols-1)/(nrows-kcols) * (1-rsquared)


def multiple_linear_regression(df,features,target,model,
                               verbose=1,cv=5,test_size=0.3):
    """ Multiple Linear Regression Modelling using given model.
    
    Depends:
    Depends on function adjusted r-squared.
    
    
    Returns:
    rmse, r2_train, ar2_train, r2_test, ar2_test, cv
    """
    
    # train test split
    train, test = train_test_split(df, test_size=0.2, random_state=100)

    # train test values
    X = df[features].values
    y = df[target].values.ravel()

    Xtrain = train[features].values
    ytrain = train[target].values.ravel()

    Xtest = test[features].values
    ytest = test[target].values.ravel()
    
    # fitting
    model.fit(Xtrain,ytrain)

    # prediction
    ypreds = model.predict(Xtest)

    # metrics
    rmse = np.sqrt(mean_squared_error(ytest,ypreds)).round(3)
    r2_train = model.score(Xtrain, ytrain).round(3)
    r2_test = model.score(Xtest, ytest).round(3)

    cv = cross_val_score(model, X, y, cv=5,n_jobs=-1,
                         verbose=verbose).mean().round(3)

    ar2_train = adjustedR2(model.score(Xtrain,ytrain),
                           Xtrain.shape[0],
                           len(features)).round(3)
    ar2_test  = adjustedR2(model.score(Xtest,ytest),
                           Xtest.shape[0] ,
                           len(features)).round(3)
    
    return (rmse, r2_train, ar2_train, r2_test, ar2_test, cv)


df_eval = pd.DataFrame({'Model': [],
                           'Details':[],
                           'Root Mean Squared Error (RMSE)':[],
                           'R-squared (training)':[],
                           'Adjusted R-squared (training)':[],
                           'R-squared (test)':[],
                           'Adjusted R-squared (test)':[],
                           '5-Fold Cross Validation':[]})

# Load the data

In [3]:
df = pd.read_csv('../data/raw/kc_house_data.csv')
df.drop(['id','date'],axis=1,inplace=True)
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


# Random Forest Modelling

In [4]:
target = ['price']
features = df.drop(['price'],axis=1).columns

In [5]:
model = RandomForestRegressor(n_estimators= 50,random_state=SEED)

rmse, r2_train, ar2_train, r2_test, ar2_test, cv = \
    multiple_linear_regression(df, features, target,model,
                               verbose=0,test_size=0.2)


df_eval.loc[len(df_eval)] = ['Random Forest Regressor',
                             '', rmse,r2_train,ar2_train,
                             r2_test,ar2_test,cv]


df_eval

,Model,Details,Root Mean Squared Error (RMSE),R-squared (training),Adjusted R-squared (training),R-squared (test),Adjusted R-squared (test),5-Fold Cross Validation
0,Random Forest Regressor,,125316.711,0.981,0.981,0.883,0.883,0.872


# Grid Search

Most important hyperparameters of Random Forest:

- n_estimators = n of trees
- max_features = max number of features considered for splitting a node
- max_depth = max number of levels in each decision tree
- min_samples_split = min number of data points placed in a node before the node is split
- min_samples_leaf = min number of data points allowed in a leaf node
- bootstrap = method for sampling data points (with or without replacement)

In [6]:
# train test split
train, test = train_test_split(df, test_size=0.2, random_state=SEED)

# train test values
X = df[features].values
y = df[target].values.ravel()

Xtrain = train[features].values
ytrain = train[target].values.ravel()

Xtest = test[features].values
ytest = test[target].values.ravel()

In [7]:
%%time
from sklearn.model_selection import GridSearchCV

model = RandomForestRegressor(random_state=SEED,n_jobs=-1)


param_grid = [
{'n_estimators': np.arange(20,100,10),
 'max_features': ['sqrt', 'auto', len(features)], 
 'max_depth': [10, 50, None],
 'min_samples_split': [2,5,10],
 'min_samples_leaf': [1,2,4],
 'bootstrap': [True, False]}
]

param_grid = [{
 'n_estimators': [500]
}]

grid_search_forest = GridSearchCV(model,
                                  param_grid,
                                  cv=5,
                                  n_jobs=-1,
                                  scoring='neg_mean_squared_error',
                                  verbose=0)

grid_search_forest.fit(Xtrain, ytrain)
# this takes very long time, comment this after use
# save the pickled result and load the pickle.

CPU times: user 1min 15s, sys: 474 ms, total: 1min 15s
Wall time: 1min 57s


GridSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=-1, random_state=100),
             n_jobs=-1, param_grid=[{'n_estimators': [500]}],
             scoring='neg_mean_squared_error')

In [8]:
import pickle

with open('../models/grid_search_rf.pkl','wb') as fo:
    pickle.dump(grid_search_forest, fo)

# Use the best parameters from grid search

In [9]:
show_methods(grid_search_forest)

,0,1,2,3,4,5,6
0,best_estimator_,cv,fit,n_features_in_,predict,refit_time_,scoring
1,best_index_,cv_results_,get_params,n_jobs,predict_log_proba,return_train_score,set_params
2,best_params_,decision_function,iid,n_splits_,predict_proba,score,transform
3,best_score_,error_score,inverse_transform,param_grid,refit,scorer_,verbose
4,classes_,estimator,multimetric_,pre_dispatch,,,


In [10]:
df_gs_rf = pd.DataFrame(grid_search_forest.cv_results_)
df_gs_rf.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_n_estimators', 'params', 'split0_test_score',
       'split1_test_score', 'split2_test_score', 'split3_test_score',
       'split4_test_score', 'mean_test_score', 'std_test_score',
       'rank_test_score'],
      dtype='object')

In [11]:
# df_gs_rf param_max_features

In [14]:
# df_gs_rf.pivot_table(
#     index=['param_max_features','param_n_estimators'],
#     columns=['param_bootstrap','param_max_depth'],
#     values='mean_test_score').round(2)\
# .style.background_gradient('Blues',axis=None)

In [15]:
grid_search_forest.best_params_

{'n_estimators': 500}

In [16]:
grid_search_forest.best_estimator_

RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=100)

In [17]:
model = RandomForestRegressor(n_estimators= 40,random_state=SEED,
                max_features=18, max_depth=50, bootstrap=True)

rmse, r2_train, ar2_train, r2_test, ar2_test, cv = \
    multiple_linear_regression(df, features, target,model,
                               verbose=0,test_size=0.2)

df_eval.loc[len(df_eval)] = ['Random Forest Regressor after grid search',
                             '', rmse,r2_train,ar2_train,
                             r2_test,ar2_test,cv]


df_eval.sort_values('Adjusted R-squared (test)')

,Model,Details,Root Mean Squared Error (RMSE),R-squared (training),Adjusted R-squared (training),R-squared (test),Adjusted R-squared (test),5-Fold Cross Validation
0,Random Forest Regressor,,125316.711,0.981,0.981,0.883,0.883,0.872
1,Random Forest Regressor after grid search,,125118.269,0.981,0.981,0.884,0.883,0.872


# Randomized search

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

# Number of trees in  forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 5)]

# max features
max_features = ['auto', 'sqrt']

# max depth of leaves
max_depth = [int(x) for x in np.linspace(1, 45, num = 3)]

# min samples split
min_samples_split = [5, 10]

# random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}


random_grid = {'n_estimators': [500]}

pprint(random_grid)

{'n_estimators': [500]}


In [21]:
rf_random = RandomizedSearchCV(estimator = model,
                               param_distributions = random_grid,
                               n_iter = 10,
                               cv = 5,
                               verbose=2,
                               random_state=SEED,
                               n_jobs = -1,
                               scoring='neg_mean_squared_error')
# Fit the random search model
rf_random.fit(Xtrain, ytrain)

/Users/poudel/opt/miniconda3/envs/dataSc/lib/python3.7/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.9min finished


RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(max_depth=50,
                                                   max_features=18,
                                                   n_estimators=40,
                                                   random_state=100),
                   n_jobs=-1, param_distributions={'n_estimators': [500]},
                   random_state=100, scoring='neg_mean_squared_error',
                   verbose=2)

In [22]:
rf_random.best_params_

{'n_estimators': 500}

In [23]:
model = RandomForestRegressor(n_estimators= 155,
                              random_state=SEED,
                              max_features='auto',
                              max_depth=23,
                              min_samples_split=10)

rmse, r2_train, ar2_train, r2_test, ar2_test, cv = \
    multiple_linear_regression(df, features, target,model,
                               verbose=0,test_size=0.2)


df_eval.loc[len(df_eval)] = ['Random Forest Regressor after grid search',
                             '', rmse,r2_train,ar2_train,
                             r2_test,ar2_test,cv]

df_eval.sort_values('Adjusted R-squared (test)')

,Model,Details,Root Mean Squared Error (RMSE),R-squared (training),Adjusted R-squared (training),R-squared (test),Adjusted R-squared (test),5-Fold Cross Validation
2,Random Forest Regressor after grid search,,126117.377,0.960,0.960,0.882,0.882,0.874
0,Random Forest Regressor,,125316.711,0.981,0.981,0.883,0.883,0.872
1,Random Forest Regressor after grid search,,125118.269,0.981,0.981,0.884,0.883,0.872


# Feature Importance

In [24]:
importances = rf_random.best_estimator_.feature_importances_
importances

array([0.00336207, 0.0095678 , 0.25321646, 0.0138223 , 0.00199991,
       0.03340136, 0.01121503, 0.00305515, 0.32698117, 0.02165846,
       0.00540681, 0.02714913, 0.00198656, 0.01485976, 0.1570932 ,
       0.0693733 , 0.03159132, 0.01426019])

In [25]:
df_imp = pd.DataFrame({'feature': features,
                      'importance': importances})

df_imp.sort_values('importance', ascending=False)\
  .style.background_gradient(subset=['importance'])

,feature,importance
8,grade,0.326981
2,sqft_living,0.253216
14,lat,0.157093
15,long,0.069373
5,waterfront,0.033401
16,sqft_living15,0.031591
11,yr_built,0.027149
9,sqft_above,0.021658
13,zipcode,0.014860
17,sqft_lot15,0.014260


# Feature Importance using eli5

In [26]:
import eli5
from eli5.sklearn import PermutationImportance
from eli5 import show_prediction

/Users/poudel/opt/miniconda3/envs/dataSc/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/poudel/opt/miniconda3/envs/dataSc/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [27]:
perm = PermutationImportance(model, random_state=1).fit(Xtest, ytest)

eli5.show_weights(perm, feature_names = features.values.tolist())

Weight,Feature
0.3622 ± 0.0170,lat
0.2656 ± 0.0125,sqft_living
0.2399 ± 0.0115,grade
0.1727 ± 0.0040,long
0.0285 ± 0.0042,yr_built
0.0276 ± 0.0022,waterfront
0.0233 ± 0.0030,sqft_living15
0.0134 ± 0.0020,zipcode
0.0089 ± 0.0009,sqft_above
0.0068 ± 0.0010,view


In [28]:
eli5.explain_weights_df(perm, feature_names=features.values.tolist())\
  .style.background_gradient(subset=['weight'])

,feature,weight,std
0,lat,0.362158,0.008484
1,sqft_living,0.265606,0.006236
2,grade,0.239867,0.005755
3,long,0.172746,0.002020
4,yr_built,0.028458,0.002106
5,waterfront,0.027558,0.001078
6,sqft_living15,0.023341,0.001480
7,zipcode,0.013418,0.000977
8,sqft_above,0.008852,0.000448
9,view,0.006840,0.000507


In [29]:
test.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
19836,285000.0,3,2.5,2437,5136,2.0,0,0,3,7,2437,0,2011,0,98002,47.3517,-122.210,2437,4614
10442,239950.0,3,2.5,1560,4800,2.0,0,0,4,7,1560,0,1974,0,98001,47.2653,-122.285,1510,12240
20548,460000.0,3,2.5,2390,47480,2.0,0,0,3,9,2390,0,2007,0,98058,47.4517,-122.084,1720,44866
11014,397500.0,3,1.0,1480,5100,1.5,0,0,3,7,1480,0,1938,1959,98103,47.6915,-122.348,1300,5100
4138,545000.0,4,3.5,1880,1341,3.0,0,0,3,8,1650,230,2007,0,98122,47.6053,-122.306,1740,1883


In [30]:
eli5.show_prediction(model, test.iloc[0,1:],show_feature_values=True)

# Time Taken

In [31]:
time_taken = time.time() - time_start_notebook
h,m = divmod(time_taken,60*60)
print('Time taken to run whole notebook: {:.0f} hr '\
      '{:.0f} min {:.0f} secs'.format(h, *divmod(m,60)))

Time taken to run whole notebook: 0 hr 16 min 59 secs
